In [30]:
import pandas as pd

In [31]:
texts = pd.read_csv("./data/posts.tsv", sep="\t", encoding="utf-8").text

In [32]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import re

In [42]:
stops = set(stopwords.words("english") + stopwords.words("russian"))

In [64]:
def post_to_words(text):
    lower_words = re.sub(ur"[^а-я]", " ", BeautifulSoup(text).get_text().lower()).split()
    meaningful_words = [w for w in lower_words if not w in stops]
    stemmer = nltk.stem.snowball.RussianStemmer()
    return [stemmer.stem(t) for t in meaningful_words]

In [78]:
processed_texts = map(lambda x: post_to_words(unicode(x)), texts)

In [83]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(processed_texts)
print 'Original: {}'.format(dictionary)
dictionary.filter_extremes(no_below = 5, no_above = 0.5, keep_n=None)
dictionary.save('nips.dict')
print 'Filtered: {}'.format(dictionary)

Original: Dictionary(3434 unique tokens: [u'\u0441\u043e\u043c\u043d\u0438\u0442\u0435\u043b\u044c\u043d', u'\u0447\u0435\u0447\u043d', u'\u043e\u0431\u043b\u0430\u0432', u'\u0441\u043f\u0438\u0441\u043e\u043a', u'\u0441\u043c\u0435\u044f']...)
Filtered: Dictionary(563 unique tokens: [u'\u043a\u043e\u0433', u'\u0441\u0441\u044b\u043b\u043a', u'\u0443\u0440\u0430\u043b\u044c\u0441\u043a', u'\u0441\u043f\u0438\u0441\u043e\u043a', u'\u043d\u0430\u0445\u0438\u043c\u043e\u0432\u0441\u043a']...)


In [84]:
corpus = [dictionary.doc2bow(text) for text in processed_texts]
corpora.MmCorpus.serialize('nips.mm', corpus) # store on disc

In [92]:
print corpus[7][0:20]

[(43, 1), (117, 1), (204, 1), (284, 1), (462, 1), (523, 1)]


In [97]:
from time import time
start = time()
model = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=5, chunksize=2000, update_every=1, passes=40, decay=0.5)
print 'Evaluation time: {}'.format((time()-start) / 60)

Evaluation time: 0.734800350666


In [105]:
for position in range(10):
    for topic in range(5):
        print model.show_topic(topic)[position][1].center(20, ' '),
    print

         ул                 встреч                 эт                  куб                  рубл        
         д                  команд               котор               навальн                дел         
      проспект              район                 год                волонтер                эт         
       выход                приход                сво                 команд                год         
        метр                 нам                навальн                наш                 путин        
       площад              навальн                росс               социолог               млрд        
        улиц                 друз                 суд                  нам                  сво         
      бульвар                штаб                 сам                 помога                нача        
        куб                  наш                 стран              инструктаж             вопрос       
         тц                  хот                  наш  

1. Вы первой теме можно заметить много слов, относящихся к местоположению: улица, проспект, метро, площадь, а также куб. Эта тема — со времен активной агитации за Навального во время выборов мэра, когда по всей Москве стояли агитационные точки — кубы.
2. Эта тема схожа с первой, но больше относится с активизации и рекрутингу волонтеров для работы в штабе и вцелом команде Навального на тех же выборах.
3. Тут получилась довольно расплывчатая тема Навального, его суды и судьбы России.
4. Эта тема опять связана с поиском волонтеров, но уже в основном после выборов мэра. Тут волонтеры ищутся для различных пикетов и проведения социологических опросов по разным темам.
5. Тема корупции и хищений миооиардов долларов друщьями Путина.